In [1]:
import pandas as pd
from graphdatascience import GraphDataScience
from getpass import getpass
from time import perf_counter
import kmedoids
import urllib.request
import zipfile

# Connect to Neo4j

The connection URI for your Neo4j instance is available in the AuraDS console or by copying the Bolt port from the details panel for for your DBMS in Neo4j Desktop. If you are using Neo4j Desktop, make sure you have GDS installed.

In [3]:
connection_uri = "neo4j+s://80102ec4.databases.neo4j.io"

In [2]:
neo4j_pwd = getpass("Neo4j password")

Neo4j password ········


In [4]:
gds = GraphDataScience(connection_uri, auth=("neo4j", neo4j_pwd))

In [5]:
gds.set_database("neo4j")

# Download datasets
## Smaller genetic dataset
Data from Cho, Ara and Shin, Junha and Hwang, Sohyun and Kim, Chanyoung and Shim, Hongseok and Kim, Hyojin and Kim, Hanhae and Lee, Insuk (2014). "WormNet v3: a network-assisted hypothesis-generating server for Caenorhabditis elegans." Nucleic acids research, 42(W1), W76-W82.

Hosted by [The Network Data Repository with Interactive Graph Analytics and Visualization](https://networkrepository.com)

In [6]:
file_name, headers = urllib.request.urlretrieve("https://nrvis.com/download/data/bio/bio-CE-LC.zip", "bio-CE-LC.zip")

In [7]:
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall('.')

## Larger collaboration dataset
Data from Gehrke, Johannes and Ginsparg, Paul and Kleinberg Jon (2003). "Overview of the 2003 KDD Cup." ACM SIGKDD Explorations Newsletter, 5(2), 149-151.

Hosted by [The Network Data Repository with Interactive Graph Analytics and Visualization](https://networkrepository.com)

In [8]:
file_name, headers = urllib.request.urlretrieve("https://nrvis.com/download/data/soc/ca-HepPh.zip", "ca-HepPh.zip")

In [9]:
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall('.')

# CLARANS functions

The k-medoid algorithm assumes that we can calculate a distance from each node in the base graph to each medoid. This means that there must be a path from each node in the graph to each other node in the graph. If you have a disconnected graph, running weakly connected components (WCC) is a good first step towards clustering. Depending on the results of WCC, you can use other clustering algorithms like CLARANS to further analyze the structure of the largest WCC components.

This function creates a subgraph based on the largest connected component in the provided graph projection. It also applies a new label to the nodes in the largest component so that we can efficiently query these nodes with Cypher.

In [10]:
def find_giant_component(projection, wccProperty, subgraph_name, subgraph_node_label):
    # Mutate the graph with wcc id so we can use it for filtering
    gds.wcc.mutate(projection, mutateProperty=wccProperty)
    # Write the wcc id to the graph so we can use it for filtering in Bloom
    gds.graph.nodeProperties.write(projection, wccProperty)
    # Get the id of the giant component
    wcc_df = gds.graph.nodeProperties.stream(projection, wccProperty)
    biggest_component_id = wcc_df['propertyValue'].value_counts().idxmax()
    # Filter the projection to the giant component
    subgraph, result = gds.graph.filter(subgraph_name,
                                projection,
                                f"n.{wccProperty} = {biggest_component_id}",
                                "*")
    gds.run_cypher(f"""MATCH (n) WHERE n.wccId = $biggestComponentId
                       CALL apoc.create.addLabels(n, [$subgraphNodeLabel])
                       YIELD node
                       RETURN count(*) AS nodesUpdated""",
                  {"biggestComponentId": biggest_component_id,
                  "subgraphNodeLabel": subgraph_node_label})
    return subgraph, result, biggest_component_id

This function selects a random group of k nodes as medoids to begin an iteration of the CLARANS search.

In [11]:
def create_new_candidate(k, label):
    candidate_df = gds.run_cypher(f"""MATCH (g:{label})
                                     WITH g ORDER BY rand()
                                     LIMIT $k
                                     WITH collect(g) AS medoids
                                     CREATE (c:Candidate)
                                     WITH c, medoids
                                     UNWIND medoids as m
                                     MERGE (c)-[:HAS_MEDOID]->(m)
                                     RETURN id(c) AS candidate_id
                                     LIMIT 1""",
                                  {"k":k, "label": label})
    return candidate_df.iloc[0,0]

This function uses the [Delta-Stepping Single-Source Shortest Path](https://neo4j.com/docs/graph-data-science/current/algorithms/delta-single-source/) algorithm to calculate the distance from each medoid to all the other nodes in the graph. We store the calculated distances as properties on `HAS_DISTANCE` relationships in the graph.

Once the distances are known, we can calculate the sum of the distances from each node in the base graph to its closest medoid.

In [12]:
def calculate_candidate_cost(projection, candidate_id):
    #If distances to all other nodes are not already present for each medoid, calculate the missing distances
    gds.run_cypher("""
                      MATCH (c)-[:HAS_MEDOID]->(m)
                      WHERE id(c) = $candidateId
                      AND NOT EXISTS {(m)-[:HAS_DISTANCE]->()}
                      WITH m
                      CALL gds.allShortestPaths.delta.write(
                          $graphName,
                          {writeRelationshipType: "HAS_DISTANCE",
                          sourceNode: id(m)})
                      YIELD relationshipsWritten
                      RETURN relationshipsWritten
                          """,
                   {"candidateId": candidate_id,
                    "graphName": projection.name()})
    #Calculate the cost for this group of candidate medoids
    cost_df = gds.run_cypher("""
                      MATCH (c)-[:HAS_MEDOID]->(m)
                      WHERE id(c) = $candidateId
                      MATCH (m)-[d:HAS_DISTANCE]->(n)
                      WITH c, m, n, d 
                      ORDER BY d.totalCost ASC
                      WITH c, n, collect(d.totalCost) AS costList
                      WITH c, n, costList[0] AS closestMedoidCost
                      WITH c, sum(closestMedoidCost) AS totalCost
                      SET c.totalCost = totalCost
                      RETURN totalCost""",
                  {"candidateId": candidate_id})
    return cost_df.iloc[0,0]

Given an existing node in the candidate graph, this function creates a neighboring candidate node by swapping out one of the medoids associated with the original candidate with a randomly selected node in the base graph. We create a `HAS_NEIGHBOR` relationship from the original candidate to the new candidate so that we have an audit trail of the candidates that were searched.

In [13]:
def create_neighboring_candidate(candidate_id, k, label):
    new_candidate_df = gds.run_cypher(f"""
                                       MATCH (c:Candidate)
                                       WHERE id(c) = $candidateId
                                       MATCH (g:{label}) 
                                       WHERE NOT EXISTS {{(c)-[:HAS_MEDOID]->(g)}}
                                       WITH c, g 
                                       ORDER BY rand()
                                       LIMIT 1
                                       MATCH (c)-[:HAS_MEDOID]->(m)
                                       WITH c, g, m 
                                       ORDER BY rand()
                                       LIMIT $k -1
                                       WITH c, g, collect(m) AS medoids
                                       WITH c, medoids + g AS newMedoids
                                       CREATE (newCandidate:Candidate)
                                       WITH c, newCandidate, newMedoids
                                       MERGE (c)-[:HAS_NEIGHBOR]->(newCandidate)
                                       WITH newMedoids, newCandidate
                                       UNWIND newMedoids as newMedoid
                                       MERGE (newCandidate)-[:HAS_MEDOID]->(newMedoid)
                                       RETURN id(newCandidate) AS newCandidateId
                                       LIMIT 1""",
                                     {"candidateId": candidate_id,
                                      "k": k,
                                      "label": label})
    return new_candidate_df.iloc[0, 0]

This function runs the full CLARANS algorithm.

In [14]:
def run_clarans(projection, k, label, num_local=2, max_neighbor=None, verbose=False):
    #If max_neighbor was not provided, calculate 1.25% times k(n-k)
    if max_neighbor == None:
        node_count_df = gds.run_cypher(f"""MATCH (n:{label}) RETURN count(*) as nodeCount""")
        node_count = node_count_df.iloc[0,0]
        max_neighbor = round(k*(node_count-k)*0.0125)
        if verbose:
            print(f"max_neighbor: {max_neighbor}")
    min_cost = 999999999
    # The i counter keeps track of the number of times we begin the search at a new randomly selected candidate node
    i = 1
    while i <= num_local:
        current_candidate_id = create_new_candidate(k, label)
        current_cost = calculate_candidate_cost(projection, current_candidate_id)
        if verbose:
            print(f"Starting search from candidate {current_candidate_id} with cost {current_cost}")
        # The j counter keeps track of the number of times we have checked a neighbor of the currently selected node
        j = 1
        while j <= max_neighbor:
            neighbor_candidate_id = create_neighboring_candidate(current_candidate_id, k, label)
            neighbor_cost = calculate_candidate_cost(projection, neighbor_candidate_id)
            if verbose:
                print(f"Neighbor cost: {neighbor_cost}")
            if neighbor_cost < current_cost:
                j = 1
                current_candidate_id = neighbor_candidate_id
                current_cost = neighbor_cost
                if verbose:
                    print(f"Current cost: {current_cost}")
            else:
                j = j + 1
        if current_cost < min_cost:
            min_cost = current_cost
            best_candidate_id = current_candidate_id
            if verbose:
                print(f"Min cost: {min_cost}")
        i = i + 1
    return best_candidate_id, min_cost

# PAM Functions
To compare perfromance of CLARANS and PAM, we need a function to provide the input distance array for the PAM algorithm.

In [ ]:
def get_distance_array(projection, 
                       relationshipWeightProperty=None, 
                       concurrency=4):
    all_pairs = gds.allShortestPaths.stream(projection, 
                                            relationshipWeightProperty = relationshipWeightProperty, 
                                            concurrency=concurrency)
    distance_df = all_pairs.pivot(index='sourceNodeId', columns='targetNodeId', values='distance')
    distance_array = distance_df.fillna(0).values
    return distance_array, distance_df.columns

# Try on small genetic network
## Load data to Neo4j

In [15]:
gds.run_cypher("""CREATE CONSTRAINT geneId FOR (p:Gene) REQUIRE p.id IS NODE KEY""")

""


In [16]:
edge_df = pd.read_csv('bio-CE-LC.edges', sep=' ', header=None)
edge_list = edge_df.values.tolist()

In [17]:
gds.run_cypher("""
UNWIND $edgeList AS rel
WITH rel[0] AS id1, rel[1] AS id2 
MERGE (m1:Gene {id:id1})
MERGE (m2:Gene {id:id2})
MERGE (m1)-[:RELATES_TO]->(m2)""",
               {"edgeList": edge_list})

""


## Create GDS projection

In [18]:
g_gene, result = gds.graph.project("genes", "Gene", {"RELATES_TO": {"orientation": "UNDIRECTED"}})

In [19]:
g_gene_connected, result, biggest_component_id = find_giant_component(g_gene, "wccId", "gene_connected", "ConnectedGene")

In [20]:
result

fromGraphName                  genes
nodeFilter               n.wccId = 0
relationshipFilter                 *
graphName             gene_connected
nodeCount                        993
relationshipCount               2600
projectMillis                     62
Name: 0, dtype: object

## Find medoids with CLARANS 

In [21]:
start_time = perf_counter()
best_candidate_id, min_cost = run_clarans(g_gene_connected, 5, "ConnectedGene")
end_time = perf_counter()
print(f"Elapsed seconds: {end_time - start_time}")

Elapsed seconds: 248.6279336229927


In [22]:
print(best_candidate_id, min_cost)

1970 3837.0


## Find medoids with PAM

In [25]:
start_time = perf_counter()
dist_array, array_index = get_distance_array(g_gene_connected)
km = kmedoids.KMedoids(5, method="fasterpam")
c = km.fit(dist_array)
end_time = perf_counter()
print(f"Elapsed seconds: {end_time - start_time}")

Elapsed seconds: 67.33841312400182


In [26]:
medoid_ids = [array_index[i] for i in c.medoid_indices_]
pam_candidate_df = gds.run_cypher("""UNWIND $medoidIds as medoidId
                                     MATCH (n) WHERE id(n) = medoidId
                                     WITH collect(n) AS medoids
                                     CREATE (c:Candidate)
                                     WITH c, medoids
                                     UNWIND medoids as m
                                     MERGE (c)-[:HAS_MEDOID]->(m)
                                     RETURN id(c) AS candidate_id
                                     LIMIT 1""",
                                  {"medoidIds": medoid_ids})
pam_candidate_id = pam_candidate_df.iloc[0,0]

In [27]:
pam_cost = calculate_candidate_cost(g_gene_connected, pam_candidate_id)

In [28]:
print(pam_candidate_id, pam_cost)

2033 3525.0


What is the percentage improvement in the total distance with PAM vs. CLARANS?

In [29]:
(3837 - 3525)/3837

0.08131352619233777

## Clean up

In [30]:
gds.graph.drop(g_gene)
gds.graph.drop(g_gene_connected)

graphName                                                   gene_connected
database                                                             neo4j
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                              993
relationshipCount                                                     2600
configuration            {'relationshipProperties': {}, 'jobId': '72bf5...
density                                                           0.002639
creationTime                           2024-01-09T22:46:26.813185992+00:00
modificationTime                       2024-01-09T22:46:26.876123796+00:00
schema                   {'graphProperties': {}, 'nodes': {'Gene': {'wc...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'Gene': {'wc...
Name: 0, dtype: object

In [31]:
gds.run_cypher("""MATCH (n) CALL {WITH n DETACH DELETE n} IN TRANSACTIONS OF 1000 ROWS""")

""


# Try on larger collaboration network
## Load data to Neo4j

In [32]:
edge_df = pd.read_csv('ca-HepPh.mtx', sep=' ', skiprows=2, header=None)
edge_list = edge_df.values.tolist()

In [33]:
gds.run_cypher("""CREATE CONSTRAINT authorId FOR (p:Author) REQUIRE p.id IS NODE KEY""")

""


In [34]:
gds.run_cypher("""
UNWIND $edgeList AS rel
WITH rel[0] AS id1, rel[1] AS id2 
MERGE (m1:Author {id:id1})
MERGE (m2:Author {id:id2})
MERGE (m1)-[:WORKED_WITH]->(m2)""",
               {"edgeList": edge_list})

""


In [35]:
g_authors, result = gds.graph.project("authors", "Author", {"WORKED_WITH": {"orientation": "UNDIRECTED"}})

In [36]:
result

nodeProjection            {'Author': {'label': 'Author', 'properties': {}}}
relationshipProjection    {'WORKED_WITH': {'aggregation': 'DEFAULT', 'or...
graphName                                                           authors
nodeCount                                                             11204
relationshipCount                                                    235238
projectMillis                                                           138
Name: 0, dtype: object

In [37]:
g_authors_connected, result, biggest_component_id = find_giant_component(g_authors, "wccId", "authors_connected", "ConnectedAuthor")

In [38]:
result

fromGraphName                   authors
nodeFilter                  n.wccId = 0
relationshipFilter                    *
graphName             authors_connected
nodeCount                         11204
relationshipCount                235238
projectMillis                       156
Name: 0, dtype: object

The full graph is the same as the subgraph, because the full graph is a connected graph. We'll use the full graph for our testing, and drop the subgraph.

In [71]:
gds.graph.drop(g_authors_connected)

Failed to write data to connection ResolvedIPv4Address(('34.28.32.244', 7687)) (ResolvedIPv4Address(('34.28.32.244', 7687)))
Failed to write data to connection IPv4Address(('80102ec4.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.28.32.244', 7687)))


graphName                                                authors_connected
database                                                             neo4j
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                            11204
relationshipCount                                                   231960
configuration                                                           {}
density                                                           0.001848
creationTime                           2024-01-10T16:14:54.145022085+00:00
modificationTime                       2024-01-10T16:14:54.144915127+00:00
schema                   {'graphProperties': {}, 'nodes': {'Author': {'...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'Author': {'...
Name: 0, dtype: object

## Find medoids with CLARANS 

In [40]:
start_time = perf_counter()
best_candidate_id, min_cost = run_clarans(g_authors, 5, "Author")
end_time = perf_counter()
print(f"Elapsed seconds: {end_time - start_time}")

Elapsed seconds: 1380.257974052991


In [42]:
print(best_candidate_id, min_cost)

12690 31164.0


What percetage of nodes were considered as medoid candidates?

In [55]:
gds.run_cypher("""MATCH (a:Author)
                  WITH count(*) AS totalNodeCount,
                  SUM(CASE WHEN EXISTS {(a)<-[:HAS_MEDOID]-(:Candidate)} THEN 1 else 0 END) 
                      AS candidateCount
                  RETURN totalNodeCount, candidateCount, 
                  candidateCount * 100.0 / totalNodeCount AS candidatePercent""")

,totalNodeCount,candidateCount,candidatePercent
0,11204,1997,17.823991


How many unique author pairs had the shortest path calculated?

In [56]:
gds.run_cypher("""MATCH (a1:Author)-[:HAS_DISTANCE]-(a2:Author)
                  WHERE id(a1) < id(a2)
                  WITH DISTINCT a1, a2
                  RETURN COUNT(*) AS authorPairCount""")

,authorPairCount
0,20379385


How many total author pair shortest paths could have been calculated?

In [62]:
11204*(11204-1)/2

62759206.0

What percent of possible shortest paths were calculated?

In [64]:
20379385/62759206*100.0

32.472343579362686

What percent of shortest paths that were calculated were redundant because the relationships are undirected?

In [65]:
gds.run_cypher("""MATCH ()-[:HAS_DISTANCE]->() RETURN count(*) as distanceCount""")

,distanceCount
0,22374388


In [66]:
(22374388-20379385)/20379385*100.0

9.789318961293484

## Find medoids with PAM

In [43]:
start_time = perf_counter()
dist_array, array_index = get_distance_array(g_authors)
km = kmedoids.KMedoids(5, method="fasterpam")
c = km.fit(dist_array)
end_time = perf_counter()
print(f"Elapsed seconds: {end_time - start_time}")

Elapsed seconds: 8890.122705282003


Create a candidate node based on the PAM medoids.

In [44]:
medoid_ids = [array_index[i] for i in c.medoid_indices_]
pam_candidate_df = gds.run_cypher("""UNWIND $medoidIds as medoidId
                                     MATCH (n) WHERE id(n) = medoidId
                                     WITH collect(n) AS medoids
                                     CREATE (c:Candidate)
                                     WITH c, medoids
                                     UNWIND medoids as m
                                     MERGE (c)-[:HAS_MEDOID]->(m)
                                     RETURN id(c) AS candidate_id
                                     LIMIT 1""",
                                  {"medoidIds": medoid_ids})
pam_candidate_id = pam_candidate_df.iloc[0,0]

Calculate the cost for the PAM candidate.

In [46]:
pam_cost = calculate_candidate_cost(g_authors, pam_candidate_id)

In [47]:
print(pam_candidate_id, pam_cost)

13391 29333.0


What is the percentage improvement in the total distance with PAM vs. CLARANS?

In [48]:
(31164 - 29333)/31164

0.0587536901553074

What was the percentage decrease in run time for CLARANS vs. PAM?

In [70]:
(8890-1380)/8890*100.0

84.4769403824522